In [3]:
from onnx import *
from onnx import shape_inference
import numpy as np
#make model
import torch
from onnx import AttributeProto, TensorProto, GraphProto
from torch.onnx.utils import register_custom_op_symbolic

In [4]:
#制作属性
arg = helper.make_attribute("pi", 3.14)
print("\nFloat attribute:\n")

#用里面自带的print函数来显示
helper.printable_attribute(arg)


Float attribute:



'pi = 3.14000010490417'

In [5]:
#制作一个节点
#     output  = op_name [i,i,i,i](input)
#格式 “Y”  = "Conv" [attribut1,attribut2,attribut3]("X",'Y','B')
node_proto = helper.make_node(
    "Conv",#要是onnx支持的操作符，不然不行
    ["X", "W", "B"],#输入命名
    ["Y"],#输出
    name='conv1',#给操作命名
    kernel=3, stride=1, pad=1)#其他参数
print(helper.printable_node(node_proto))

%Y = Conv[kernel = 3, pad = 1, stride = 1](%X, %W, %B)


In [8]:
node_proto.attribute[0]

name: "kernel"
i: 3
type: INT

In [9]:
#查看参数
print(node_proto.attribute)#列表保存所有的参数
#这是一个列表，可进行参数修改,也就是可以遍历改变属性
node_proto.attribute[1].i = 15
node_proto.attribute[2].i = 2
print(helper.printable_node(node_proto))

[name: "kernel"
i: 3
type: INT
, name: "pad"
i: 1
type: INT
, name: "stride"
i: 1
type: INT
]
%Y = Conv[kernel = 3, pad = 15, stride = 2](%X, %W, %B)


In [11]:
#观察完整图的构建，([节点，节点，..],名称,[输入节点信息,输入节点信息,输入节点信息,...],[输出节点信息是最后一个,不是中间节点],[写一个输入])
graph_proto = helper.make_graph(
    [
        helper.make_node("FC", ["X", "W1", "B1"], ["H1"]),
        helper.make_node("Relu", ["H1"], ["R1"]),
        helper.make_node("Conv",["X"],["B"]),
        helper.make_node("FC", ["R1", "W2", "B2"], ["Y"]),
        
    ],
    "MLP",
    [
        helper.make_tensor_value_info('X', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('W1', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('B1', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('W2', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('B2', TensorProto.FLOAT, [1]),
    ],
    [
        helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1]),
    ],
    [helper.make_tensor('Pads', TensorProto.INT64, [4,], [0, 0, 1, 1,])],
)
model = helper.make_model(graph_proto)
print("\ngraph proto:\n")
helper.printable_graph(graph_proto)


graph proto:



'graph MLP (\n  %X[FLOAT, 1]\n  %W1[FLOAT, 1]\n  %B1[FLOAT, 1]\n  %W2[FLOAT, 1]\n  %B2[FLOAT, 1]\n) initializers (\n  %Pads[INT64, 4]\n) {\n  %H1 = FC(%X, %W1, %B1)\n  %R1 = Relu(%H1)\n  %B = Conv(%X)\n  %Y = FC(%R1, %W2, %B2)\n  return %Y\n}'

In [12]:
#查看图中的不同信息
#graph_proto.node
#graph_proto.input
#graph_proto.output
graph_proto.initializer

[dims: 4
data_type: 7
int64_data: 0
int64_data: 0
int64_data: 1
int64_data: 1
name: "Pads"
]

In [ ]:
# Create one input (ValueInfoProto)
X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [1, 2])

# Create second input (ValueInfoProto)
Pads = helper.make_tensor_value_info('Pads', TensorProto.INT64, [4])

# Create one output (ValueInfoProto)
Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 4])

# Create a node (NodeProto)
node_def = helper.make_node(
    'Pad', # node name
    ['X', 'Pads'], # inputs
    ['Y'], # outputs
    mode='constant', # Attributes
)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [node_def],
    "test-model",
    [X, Pads],
    [Y],
    [helper.make_tensor('Pads', TensorProto.INT64, [4,], [0, 0, 1, 1,])],
)

# 要把图做成模型才能导出
model_def = helper.make_model(graph_def,
                              producer_name='onnx-example')

#print('The producer_name in model: {}\n'.format(model_def.producer_name))
print('The graph in model:\n{}'.format(model_def.graph))
onnx.checker.check_model(model_def)
print('The model is checked!')

In [ ]:
model_def.graph

In [7]:
#导出模型后，可以可视化
onnx.save_model(model_def,'text.onnx')

In [8]:
test_model = onnx.load_model('text.onnx')

In [ ]:
#模型分为节点和初始化，注意他们是在图下面的
#test_model.graph
#test_model.graph.node
test_model.graph.initializer

In [ ]:
test_model.output

In [ ]:
# Preprocessing: create a model with two nodes, Y's shape is unknown
node1 = helper.make_node('Transpose', ['X'], ['Y'], perm=[1, 0, 2])
node2 = helper.make_node('Transpose', ['Y'], ['Z'], perm=[1, 0, 2])

graph = helper.make_graph(
    [node1, node2],
    'two-transposes',
    [helper.make_tensor_value_info('X', TensorProto.FLOAT, (2, 3, 4))],
    [helper.make_tensor_value_info('Z', TensorProto.FLOAT, (2, 3, 4))],
)

# torch.onnx


'''

        类型声明，g是图，这是一个assert的装饰器
        @parse_args('v', 'i', 'fs')
        foo(g, a, b, c):
          assert isinstance(a, torch._C.Value)
          assert isinstance(b, int)
          assert isinstance(c, list)
          assert isinstance(c[0], float)
      
        "v":不需要转换, 保持 torch._C.Value.
        "i": int
        "is": list(int)
        "f": float
        "fs": list of float
        "b": bool
        "s": str
        "t": torch.Tensor   
      
'''

# 如何导出ONNX的OP表示 

### 1 Aten库自带算子（张量可以直接使用的函数）
    这种形式：虽然有直接使用的算子函数，但是在ONNX中没有正确对应的算子，需要进行符号函数声明
    如 torch.asinh = Asinh，这样才能调用到对应算子
    用 register_op 可以为 ATen 算子补充注册符号函数
    如果在 g.op() 里不加前面的命名空间，则算子会被默认成 ONNX 的官方算子

In [ ]:
#只要写一个符号函数，符号函数一般就是函数名字
#其作用就是 1 关联函数映射  2 算子捏合 3 类型转化

#@parse_args("v", "v", "v", "i", "i", "i","i","none")声明数据类型
def asinh_symbolic(g, input, *, out=None):
    #对应ONNX的算子,在onnx中没有asinh,只有Asinh
    return g.op("Asinh", input_i=input)
#绑定，"torch.asinh"-------"Asinh"
register_op('asinh', asinh_symbolic, '', 9)
#register_op(opname, op, domain, version)

#### 在torch.onnx 库中用register_custom_op_symbolic 来注册一个op,有各式要求‘mmcv::max’，必须是这样的
#### _registry用于保存所有的自定义OP
####  {('mmcv','3.5'):{'max':max,'min':min,'test':test},('other','1.0'):{'func1':func1,'func2':func2}}

In [1]:
#用 register_custom_op_symbolic 可以为 TorchScript 算子补充注册符号函数,也就是自定义算子
from torch.onnx.symbolic_registry import is_registered_op,get_registered_op
from torch.onnx.utils import register_custom_op_symbolic
# register_custom_op_symbolic(symbolic_name, symbolic_fn, opset_version) 
#这个函数调用register_op进行操作注册，只是要求symbolic_name必须是domain::funcname的格式，
#做了简单的格式检查，和register_op是一样的
#@parse_args("v", "v", "v", "v", "v", "i", "i", "i", "i", "i", "i", "i", "i", "none")
def newfunc(g,input,max_value):
    #调用都是onnx里面支持的OP，实现包装组合执行函数，那要看到底怎么组合出这个自定义算子
    return  g.op('Constant', input, max_value)
#注册自定义算子,这是把myfunc::newfunc 注册到onnx11中去，使用g.Op（‘myfunc::newfunc'，input）这种形式
register_custom_op_symbolic('myfunc::newfunc', newfunc,11)

In [3]:
import torch
from torch import nn

In [6]:
#当没有已经存在的OP可以表示的时候
class CAWeightFunction(torch.autograd.Function):

    @staticmethod
    def symbolic(g, t, f,iou_threshold=10):
        #转化属性值为常量，因为有些内部自带的OP接受参数的数据类型是特定的，需要用这个进行转化
        #也有些自定义的不用，直接用float(iou_threshold)就可以了
        iou_threshold = g.op( 'Constant',value_t=torch.tensor([iou_threshold], dtype=torch.float))
        
        
        #即使没有任何实现也可以导出onnx格式，但是执行onnxruntime则需要调用具体函数来实现,只会取::后面部分作为OP_NAME
        #如果没有::，他会到domain ::中寻找OP，找不到就报错了
        return  g.op("myfunc::newfunc", t)
    
    #onnx并不需要实现这个，C++或者CUDA的延伸函数才是在这里使用的
    @staticmethod
    def forward(ctx, t, f,iou_threshold):
        n, c, h, w = t.size()
        weight = torch.zeros(n, h + w - 1, h, w).to(t.device)
        ctx.save_for_backward(t, f)
        #out   =  torch.ops.myfunc.func()
        return weight

    @staticmethod
    def backward(ctx, dw):
        t, f = ctx.saved_tensors
        dt = torch.zeros_like(t)
        df = torch.zeros_like(f)
        return dt, df
test= CAWeightFunction.apply
class Mymode(nn.Module):
    
    def __init__(self):
        super(Mymode,self).__init__()
        
        self.conv1 = nn.Conv2d(3,10,kernel_size=3)
        self.b = nn.Parameter(torch.randn(5,5))
        
        
        
    def forward(self,input):
        x = self.conv1(input)
        x = test(input,self.b,1.0)
        return x
        
model = Mymode()   
model.eval()
dummy_input = torch.randn(1,3,10,10)
c = torch.randn(1)
v = torch.tensor(5.0)
torch.onnx.export(model,
                 (dummy_input,),
                 'test.onnx',
                 export_params=True,
                  opset_version=11)

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 1 WARNING 0 ERROR ========================
1 WARNING were not printed due to the log level.



In [17]:
#辅助函数，不用再写复杂的模块了
class WrapFunction(nn.Module):

    def __init__(self, wrapped_function):
        super(WrapFunction, self).__init__()
        self.wrapped_function = wrapped_function

    def forward(self, *args, **kwargs):
        return self.wrapped_function(*args, **kwargs)

In [ ]:
class custom(torch.autograd.Function):
    
    @staticmethod
    def symbolic(g, inputs,value):
        #关键点！！！！！！！！！！！！
        #这里的名字是要和函数对应的，mmcv是空间，Custom是函数名
        #所有的属性名字必须和cpp文件中的一样，并且，名字中要有类型表示，比如
        #"value_add"被用于cpp文件中提取Info，并且他是一个int类型，所以这里要给 value_add_i=value_add
        #"spatial_scale"被用于cpp文件，他的类型是float，所以这里要给 spatial_scale_f=spatial_scale
        #"mode"被用于cpp文件，他是string，所以这里要用 mode_s=pool_mode
        #"aligned"被用于cpp文件，他是bool，所以这里要用aligned_i=aligned
        #总结，就是 name_type这种形式才能识别
        return g.op('mmcv::Custom',inputs,value_add_f=value)
    @staticmethod
    def forward(ctx,inputs,value):
        outputs =inputs +value
        return outputs
testfunc = custom.apply

    @parse_args('v', 'i', 'fs') 这个是对输入进行类型转换的装饰器
    foo(g, a, b, c):
    "v": 不转化, keep torch._C.Value.
    "i": int
    "is": list(int)
    "f": float
    "fs": list of float
    "b": bool
    "s": str
    "t": torch.Tensor

In [44]:

_value =1.0
#other = value
def func(inputs):
    return testfunc(inputs,_value)#other=value

In [ ]:
wrapped_model = WrapFunction(func)
wrapped_model.cpu().eval()
#初始化模型输入
inputs = torch.randn(1,3,3)
#导出onnx模型，可以考虑其他参数的使用
#导出onnx模型，可以考虑其他参数的使用
with torch.no_grad():
    torch.onnx.export(wrapped_model,
                      (inputs,),
                      'sample.onnx',
                      input_names=['inputs'])
#bug:RuntimeError: Unable to cast from non-held to held instance (T& to Holder<T>) (compile in debug mode for type information)

In [22]:
onnx_model = onnx.load('sample.onnx')

In [ ]:
#生成onnxruntime的模型
sess = rt.InferenceSession('sample.onnx', session_options)

In [ ]:
#生成onnxruntime的模型
sess = rt.InferenceSession('sample.onnx', session_options)
#获取结果，输入要和input_names对应起来
onnx_output = sess.run(None, {
    'inputs': inputs.detach().numpy(),
})

# 算子学习

In [1]:
import numpy as np

In [ ]:
def check_op(name,inputs,outputs,op,
             filename='op.onnx'):
    '''
    1v1算子检查
    '''
    graph= make_graph([make_node(
    name,
    inputs=inputs,
    outputs=outputs)],
    'test',
    [make_tensor_value_info(inputs[0], TensorProto.FLOAT, [3,3])],
    [make_tensor_value_info(outputs[0], TensorProto.FLOAT, [3,3])])
    model = make_model(graph)
    onnx.save_model(model,filename)
    sess = rt.InferenceSession(filename)
    
    inputs =torch.rand(3,3)-0.1
    onnx_output = sess.run(None, {
    'x': inputs.detach().numpy(),})
    
    return np.allclose(onnx_output,op(inputs))